## Выбор тарифа

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Постройте модель с максимально большим значением accuracy.

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [6]:
data = pd.read_csv('users_behavior.csv')

In [12]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [13]:
X = data.drop('is_ultra', axis=1)  # Признаки
y = data['is_ultra']               # Целевая переменная

# Первое разделение: тестовая выборка (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Второе разделение: обучающая (60%) и валидационная (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42
)

# Итоговые размеры:
print(f"Обучающая: {X_train.shape[0]} строк")
print(f"Валидационная: {X_val.shape[0]} строк")
print(f"Тестовая: {X_test.shape[0]} строк")

Обучающая: 1928 строк
Валидационная: 643 строк
Тестовая: 643 строк


#### Тестирование модели дерева решений для задачи классификации с разными значениями глубины дерева и минимального числа объектов в листе

In [57]:
best_acc = 0
for depth in range(1, 20):  # Увеличили до 20
    for min_samples_leaf in [1, 2, 5, 10]:  # Добавили минимальное число объектов в листе
        model = DecisionTreeClassifier(
            max_depth=depth,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        model.fit(X_train, y_train) # обучите модель на тренировочной выборке
        acc = model.score(X_val, y_val)# посчитайте качество модели на валидационной выборке
        if acc > best_acc:
            best_acc = acc
            max_depth = depth
            min_leaf = min_samples_leaf
        
print("max_depth =", max_depth, "min_leaf =", min_leaf,", accuracy ", best_acc)

max_depth = 9 min_leaf = 5 , accuracy  0.7962674961119751


#### Тестирование модели слуйчайного леса для задачи классификации с разными значениями количества деревьев и глубины дерева

In [60]:
best_acc = 0
for est in [10, 50, 100, 200]:  # Увеличили диапазон
    for depth in [5, 10, None]:  # None — без ограничения глубины
        model = RandomForestClassifier(
            n_estimators=est,
            max_depth=depth,
            random_state=42
        ) # обучите модель с заданным количеством деревьев
        model.fit(X_train, y_train) # обучите модель на тренировочной выборке
        acc = model.score(X_val, y_val)# посчитайте качество модели на валидационной выборке
        if acc > best_acc:
            best_acc = acc
            n_estimators = est
            max_depth = depth
print("n_estimators =", n_estimators,"max_depth =", max_depth,", accuracy ", best_acc)

n_estimators = 100 max_depth = 5 , accuracy  0.80248833592535


#### Тестирование модели логистической регрессии для задачи классификации

In [66]:
best_acc = 0
for C in [0.01, 0.1, 1, 10]:  # Сила регуляризации (меньше C → сильнее регуляризация)
    for solver in ['lbfgs', 'liblinear', 'saga']:
        model = LogisticRegression(
            C=C,
            solver=solver,
            max_iter=1000,
            random_state=42
        )
        model.fit(X_train, y_train)
        acc = model.score(X_val, y_val)# посчитайте качество модели на валидационной выборке
        if acc > best_acc:
            best_acc = acc
            reg = C
            solv = solver
print("inv_reg_strength =", reg,"solver =", solv,", accuracy ", best_acc)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


inv_reg_strength = 0.01 solver = lbfgs , accuracy  0.744945567651633


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


#### Проверка моделей на вменяемость. Бейзлайн модели.

In [64]:
dummy = DummyClassifier(strategy='uniform').fit(X_train, y_train)
print("Accuracy случайного угадывания:", dummy.score(X_test, y_test))

Accuracy случайного угадывания: 0.48055987558320373


In [65]:
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
print("Accuracy константного предсказания:", dummy.score(X_test, y_test))

Accuracy константного предсказания: 0.7076205287713841


#### Проверка моделей на тестовой выборке

In [62]:
# Дерево решений
model = DecisionTreeClassifier(
            max_depth=9,
            min_samples_leaf=5,
            random_state=42
        )
model.fit(X_train, y_train)
print("Accuracy модели дерева решений:", model.score(X_test, y_test))

Accuracy модели дерева решений: 0.7993779160186625


In [63]:
# Случайный лес
model = RandomForestClassifier(
            n_estimators=100,
             max_depth=5,
            random_state=42
        )
model.fit(X_train, y_train)
print("Accuracy модели случайного леса:", model.score(X_test, y_test))

Accuracy модели случайного леса: 0.8087091757387247


In [51]:
# Логистическая регрессия
model = LogisticRegression(
            C=0.01,
            solver='lbfgs',
            max_iter=1000,
            random_state=42
        )
model.fit(X_train, y_train)
print("Accuracy модели логистической регресии:", model.score(X_test, y_test))

Accuracy модели логистической регресии: 0.7589424572317263


#### Выводы:
* Данные нелинейные. Random Forest и Decision Tree показали лучший результат, потому что умеют работать со сложными зависимостями. Logistic Regression — линейная модель, и её точность ограничена.
* Random Forest — лучший выбор для этих данных. accuracy на тесте 0.809 — это хороший результат.
* Decision Tree тоже неплох, accuracy близка к Random Forest, но случайный лес устойчивее к переобучению. Decision Tree может переобучаться на шумных данных.